In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import gc

In [ ]:
import re
import torch

In [ ]:
import Levenshtein

In [ ]:
#df = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
#df
train = pd.read_csv(INPUT_DIR+'train.csv')
test = pd.read_csv(INPUT_DIR+'test.csv')

In [ ]:
# ====================================================
# CPC Data
# ====================================================
def get_cpc_texts():
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir('../input/cpc-data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        with open(f'../input/cpc-data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt') as f:
            s = f.read()
        pattern = f'{cpc}\t\t.+'
        result = re.findall(pattern, s)
        cpc_result = result[0].lstrip(pattern)
        for context in [c for c in contexts if c[0] == cpc]:
            pattern = f'{context}\t\t.+'
            result = re.findall(pattern, s)
            results[context] = cpc_result + ". " + result[0].lstrip(pattern)
    return results


cpc_texts = get_cpc_texts()
torch.save(cpc_texts, OUTPUT_DIR+"cpc_texts.pth")
train['context_text'] = train['context'].map(cpc_texts)
test['context_text'] = test['context'].map(cpc_texts)
display(train.head())
display(test.head())

In [ ]:
#set(train['context']).symmetric_difference(set(test['context']))

In [ ]:
#set(test['context']) - set(train['context'])

In [ ]:
#set(test['context'])

In [ ]:
#set(train['context']) - set(test['context'])

In [ ]:
df = train.copy()
colname = 'anchor'
plt.figure(figsize=(15,7))
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
s.plot.pie(title=colname);

In [ ]:
df = test.copy()
colname = 'anchor'
plt.figure(figsize=(15,7))
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
s.plot.pie(title=colname);

In [ ]:
df = train.copy()
colname = 'context'
plt.figure(figsize=(15,7))
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
s.plot.pie(title=colname);

In [ ]:
df = test.copy()
colname = 'context'
plt.figure(figsize=(15,7))
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
s.plot.pie(title=colname);

In [ ]:
# def chop_context_text(df):
#     df['context_text']

In [ ]:
train[colname+'_hl'] = train[colname].apply(lambda x: x[0])
test[colname+'_hl'] = test[colname].apply(lambda x: x[0])

In [ ]:
df = train.copy()
colname = 'context_hl'
plt.figure(figsize=(15,7))
#df[colname+'_hl'] = df[colname].apply(lambda x: x[0])
#colname += '_hl'
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
s.plot.pie(title=colname);

In [ ]:
df = test.copy()
colname = 'context_hl'
plt.figure(figsize=(15,7))
#df[colname+'_hl'] = df[colname].apply(lambda x: x[0])
#colname += '_hl'
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
s.plot.pie(title=colname);

In [ ]:
def get_edit_distance(df, colname):
    df[colname] = 0
    for i,(_,rows) in enumerate(df.iterrows()):
        df.loc[i, colname] = Levenshtein.distance(rows['anchor'], rows['target'])
    return df

In [ ]:
colname = 'edit_distance'
train = get_edit_distance(train, colname)
test = get_edit_distance(test, colname)

In [ ]:
df = train.copy()
plt.figure(figsize=(15,7))
df[colname].plot.hist(title=colname);

In [ ]:
df = test.copy()
plt.figure(figsize=(15,7))
df[colname].plot.hist(title=colname);

In [ ]:
df = train.copy()
df.groupby('context')['anchor'].nunique().plot.hist(bins=100);

In [ ]:
df = test.copy()
df.groupby('context')['anchor'].nunique().plot.hist(bins=100);

In [ ]:
df = train.copy()
df.groupby('context_hl')['anchor'].nunique().plot.hist(bins=100);

In [ ]:
df = test.copy()
df.groupby('context_hl')['anchor'].nunique().plot.hist(bins=100);

In [ ]:
df = train.copy()
df.groupby('context')['target'].nunique().plot.hist(bins=100);

In [ ]:
df = test.copy()
df.groupby('context')['target'].nunique().plot.hist(bins=100);

In [ ]:
df = train.copy()
df.groupby('context_hl')['target'].nunique().plot.hist(bins=100);

In [ ]:
df = test.copy()
df.groupby('context_hl')['target'].nunique().plot.hist(bins=100);

In [ ]:
df = train.copy()
df.groupby('anchor')['target'].nunique().plot.hist(bins=100);

In [ ]:
df = test.copy()
df.groupby('anchor')['target'].nunique().plot.hist(bins=100);

In [ ]:
df = train.copy()
np.sort(df['context'].unique())

In [ ]:
df = test.copy()
np.sort(df['context'].unique())

In [ ]:
df = train.copy()
for ct in np.sort(df['context'].unique()):
    print(ct, '-'*96)
    print(df.loc[df['context']==ct, 'context_text'].unique())
    print()

In [ ]:
df = train.copy()
colname = 'target'
plt.figure(figsize=(15,7))
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
# s.plot.pie(title=colname);

In [ ]:
df = test.copy()
colname = 'target'
plt.figure(figsize=(15,7))
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
# s.plot.pie(title=colname);

In [ ]:
df = train.copy()
colname = 'score'
plt.figure(figsize=(15,7))
print(df[colname].nunique())
print()
s = df[colname].value_counts()
display(s)
s.plot.pie(title=colname);

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#df.set_index('id', inplace=True)

In [ ]:
df = train.copy()
y = df['score'].values
y.shape

In [ ]:
train

In [ ]:
#cols = ['anchor', 'target', 'context', 'context_hl']
cols = ['anchor', 'target', 'context_hl', 'edit_distance']
#X = pd.get_dummies(df[cols]).values
from sklearn import preprocessing
#le = preprocessing.LabelEncoder()
le = preprocessing.OneHotEncoder(handle_unknown='ignore')

# les = []
# res = []
# for col in cols:
#     _x = le.fit_transform(df[col].values).reshape(-1, 1)
#     res.append(_x)
#     les.append(le)
# X = np.concatenate(res, axis=1)
X = le.fit_transform(df[cols].values)
#X = X.toarray()

In [ ]:
print(type(X))
X.shape

In [ ]:
y = y*100
y = y.astype('int')
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [ ]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train.toarray(), y_train).predict(X_test.toarray())

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='macro')
# 0.19586373743867452
# 0.19854370953842218 (edit_distance)

In [ ]:
f1_score(y_test, y_pred, average='micro')
# 0.19476887645994406
# 0.19751055546416627 (edit_distance)

In [ ]:
from sklearn.feature_selection import r_regression

In [ ]:
r_regression(y_test.reshape(-1, 1), y_pred)[0]

In [ ]:
del df, X, y, X_train, y_train, X_test, y_test, y_pred
gc.collect()

# Inference

In [ ]:
test = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
test

In [ ]:
# context_hl
colname = 'context'
test[colname+'_hl'] = test[colname].apply(lambda x: x[0])

# edit_distance
colname = 'edit_distance'
test = get_edit_distance(test, colname)

In [ ]:
#X_test = pd.get_dummies(test[cols]).values
X_test = le.transform(test[cols].values)

In [ ]:
y_pred = gnb.predict(X_test.toarray())

In [ ]:
test['score'] = y_pred/100
test = test[['id', 'score']]
test.tail()

In [ ]:
test.to_csv('submission.csv', index=False)

In [ ]:
#pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv')